In [1]:
import os, sys, glob

import h5py
import numpy as np
import pandas as pd
from scipy import stats
import dama as dm
from dama.utils.stats import weighted_quantile
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
params = {'figure.figsize': (7, 7*0.618),
          'legend.fontsize': 16,
          'axes.labelsize': 16,
          'axes.titlesize': 16,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16}
plt.rcParams.update(params)

path = '/mnt/research/IceCube/FLERCNN/tech_paper/hdf5/'


In [3]:
def load_file(f_name, keys, itype=['all']):
    f = h5py.File(f_name, 'r')
    
    if itype[0] == 'all':
        itype = list(f.keys())
        
    var_dict = {}
    for k in keys:
        arr = np.array([])
        for i in itype:
            arr = np.append(arr, f[i][k])
        var_dict[k] = arr
        
    arr = np.array([])
    for i in itype:
        arr = np.append(arr, np.repeat(i, len(f[i]['MCInIcePrimary.energy'])))
        var_dict['interaction'] = arr
        
    f.close()
        
    return dm.PointData(**var_dict)


def calc_ang_err(MC, azi_key='L7_reconstructed_azimuth', zen_key='L7_reconstructed_coszen'):
    azi_true, azi_reco = MC['MCInIcePrimary.dir.azimuth'], MC[azi_key]
    zen_true = np.arccos(MC['MCInIcePrimary.dir.coszen'])
    if 'coszen' in zen_key:
        zen_reco = np.arccos(MC[zen_key])
    else:
        zen_reco = MC[zen_key]
    
    true_vec = np.stack([np.sin(zen_true) * np.cos(azi_true), np.sin(zen_true) * np.sin(azi_true), np.cos(zen_true)], axis=1)
    reco_vec = np.stack([np.sin(zen_reco) * np.cos(azi_reco), np.sin(zen_reco) * np.sin(azi_reco), np.cos(zen_reco)], axis=1)
    err = []
    for i in range(len(true_vec)):
        err.append(np.dot(true_vec[i], reco_vec[i]))
    return np.arccos(np.clip(err, -1, 1))

def plot_quants(vals, color):
    plt.axvline(np.percentile(vals, 25), color=color, linestyle='--', linewidth=2)
    plt.axvline(np.median(vals), color=color, linestyle='-', linewidth=2)    
    plt.axvline(np.percentile(vals, 75), color=color, linestyle='--', linewidth=2)
    
def correct_azi(azi):
    azi = np.where(azi<-np.pi, azi+2*np.pi, azi)
    return np.where(azi>np.pi, azi-2*np.pi, azi)

In [4]:
import matplotlib.patches as mpatches

In [133]:
verification = h5py.File(path+"oscnext_santaleera_genie_0000.hdf5", 'r')
#print(verification['numu_cc'].keys())
verification.close()

In [134]:
flercnn = h5py.File(path+"oscNext_pisa_genie_0000_flercnn.hdf5", 'r')
#print(flercnn['numu_cc'].keys())
flercnn.close()

In [135]:
f_name = path+"oscnext_santaleera_genie_0000.hdf5"
keys = ['I3EventHeader.event_id', 'I3EventHeader.run_id', 'I3EventHeader.sub_run_id', 
        'MCInIcePrimary.dir.coszen', 'MCInIcePrimary.energy', 'I3MCWeightDict.OneWeight', 'I3MCWeightDict.InteractionType',
        
        'SANTA_LEERAFit_DefaultSelection_Particle.energy', 'SANTA_LEERAFit_DefaultSelection_Particle.dir.coszen',
        'GBM_PID.value', 'SANTA_LEERAFit_result_dict.SuccessfulFit', 'L5_oscNext_bool.value', 'SRTTWOfflinePulsesDC_ContainmentVars.n_outer',
        'SRTTWOfflinePulsesDC_ContainmentVars.z_travel_top15','SANTA_sel_reduced_chisquare.value','L4_MuonClassifier_Data_ProbNu.value'
       ]
MC_santa = load_file(f_name, keys, itype=['numu_cc'])
print(f'Loaded {MC_santa.size} events')

Loaded 1313174 events


In [136]:
#cuts = (reco_z > -495.) & (reco_z < -225.) & (reco_rho < 200.) & ( L7_CoincidentMuon_Variables.n_top15 < 0.5 ) & ( L7_CoincidentMuon_Variables.n_outer < 7.5 ) & (L5_SANTA_DirectPulsesHitMultiplicity.n_hit_doms > 2.5) & (nDOM >=7) & (l7_muon_classifier_prob_nu >=0.8)
# & (reco_energy >= 5.) & (reco_energy <= 100.) & (reco_coszen <= 0.3)

flercnn = path+"oscNext_pisa_genie_0000_flercnn.hdf5"
keys = [ 'I3MCWeightDict.OneWeight', 'I3MCWeightDict.InteractionType',
        'I3EventHeader.event_id', 'I3EventHeader.run_id',  'I3EventHeader.sub_run_id', 
        'MCInIcePrimary.dir.coszen', 'MCInIcePrimary.energy', 'MCInIcePrimary.pos.z', 'MCExtraTruthInfo.vertex_rho36',
        'MCInIcePrimary.pos.x', 'MCInIcePrimary.pos.y', 'MCExtraTruthInfo.vertex_rho36',
    
        "FLERCNN_vertex_x", "FLERCNN_vertex_y", "FLERCNN_vertex_z", "FLERCNN_BDT_ProbNu",
        "FLERCNN_vertex_rho36", "FLERCNN_coszen", 
        "FLERCNN_energy", 'FLERCNN_nDOM', 'FLERCNN_prob_muon_v3', 'FLERCNN_prob_nu', 'FLERCNN_prob_track',
        'L4_MuonClassifier_Data_ProbNu',
        'L7_CoincidentMuon_Variables.z_travel_top15', 'L7_CoincidentMuon_bool', 'L7_NarrowCorridorCutPulsesHitStatistics.z_min',
        'L7_CoincidentMuon_Variables.n_outer', 'L7_CoincidentMuon_Variables.n_top15'
       ]
MC_flercnn = load_file(flercnn, keys,itype=['numu_cc'])
print(f'Loaded {MC_flercnn.size} events')

Loaded 2845992 events


In [137]:
#cuts = (l4_muon_classifier_prob_nu > 0.97) & (santa_chi2dof < 50) & (leera_success >= 1) & (n_outer < 8) & (z_travel_top15 >= 0)
#keys:
'''           'reco_energy':'SANTA_LEERAFit_DefaultSelection_Particle.energy',
             'reco_coszen':'SANTA_LEERAFit_DefaultSelection_Particle.dir.coszen',
             'pid':'GBM_PID.value',
             'l5_cut': 'L5_oscNext_bool.value',
             'leera_success': 'SANTA_LEERAFit_result_dict.SuccessfulFit',
             'n_outer': 'SRTTWOfflinePulsesDC_ContainmentVars.n_outer',
             'z_travel_top15': 'SRTTWOfflinePulsesDC_ContainmentVars.z_travel_top15',
             'santa_chi2dof': 'SANTA_sel_reduced_chisquare.value',
             'l4_muon_classifier_prob_nu': 'L4_MuonClassifier_Data_ProbNu.value',
'''
cuts_santa = (MC_santa['L4_MuonClassifier_Data_ProbNu.value'] > 0.97) & (MC_santa['SANTA_sel_reduced_chisquare.value'] < 50) & (MC_santa['SANTA_LEERAFit_result_dict.SuccessfulFit']>=1) \
            & (MC_santa['SRTTWOfflinePulsesDC_ContainmentVars.n_outer']<8) & (MC_santa['SRTTWOfflinePulsesDC_ContainmentVars.z_travel_top15'] >=0)


In [138]:
PD_SANTA = dm.PointData({
    'zen_true': MC_santa['MCInIcePrimary.dir.coszen'],
    'zen_reco': MC_santa['SANTA_LEERAFit_DefaultSelection_Particle.dir.coszen'],
    'zen_err': MC_santa['MCInIcePrimary.dir.coszen']-MC_santa['SANTA_LEERAFit_DefaultSelection_Particle.dir.coszen'],
    'energy_true': MC_santa['MCInIcePrimary.energy'],
    'energy_reco': MC_santa['SANTA_LEERAFit_DefaultSelection_Particle.energy'],
    'weights': MC_santa['I3MCWeightDict.OneWeight'],
    'interaction': MC_santa['I3MCWeightDict.InteractionType'],
    'pid': MC_santa['GBM_PID.value'],
    'event_id':MC_santa['I3EventHeader.event_id'],
    })[cuts_santa]

In [159]:
mask_ind = []
santa_ind = []
for santa_i, (zen, e) in enumerate(zip(PD_SANTA['zen_true'], PD_SANTA['energy_true'])):
    ind = np.where(np.logical_and((MC_flercnn['MCInIcePrimary.dir.coszen']==zen), (MC_flercnn['MCInIcePrimary.energy']==e)))
    if len(ind[0]) == 1: 
        mask_ind.append(ind[0][0])
        santa_ind.append(santa_i)
    else:
        print(ind, zen, e, santa_i, PD_SANTA['event_id'][santa_i], np.where(MC_flercnn['I3EventHeader.event_id']==PD_SANTA['event_id'][santa_i]))

(array([], dtype=int64),) -0.2968401190950671 2.747761882986414 186283 1835.0 (array([  39505,   67808,   75488,   86512,  103517,  110922,  137182,
        141041,  176459,  249678,  410733,  412677,  423986,  427688,
        429470,  477832,  586924,  637516,  678409,  744559,  796971,
        901889,  905597,  946797,  993527, 1001018, 1012335, 1021792,
       1027291, 1126579, 1160253, 1162096, 1207007, 1223708, 1248033,
       1257419, 1370065, 1426398, 1439435, 1467637, 1507140, 1538785,
       1542530, 1559343, 1568845, 1574512, 1613618, 1649075, 1650914,
       1684864, 1722180, 1769019, 1810017, 1815636, 1849483, 1860840,
       1905790, 1932084, 1937566, 1978728, 2004676, 2089400, 2093137,
       2108258, 2132945, 2136720, 2189603, 2204612, 2264502, 2335918,
       2420406, 2478324, 2510289, 2555374, 2559169, 2587148, 2636053,
       2753839, 2759486]),)
(array([], dtype=int64),) -0.6389413477536445 1.88666691439136 186284 3605.0 (array([   4017,   15307,   64163,  111046,  1

In [ ]:
common_events = MC_flercnn[mask_ind]
common_in_santa = MC_santa[santa_ind]
#zmask = pd.DataFrame(MC_flercnn['MCInIcePrimary.dir.coszen']).isin(list(PD_SANTA['zen_true']))
#emask = pd.DataFrame(MC_flercnn['MCInIcePrimary.energy']).isin(list(PD_SANTA['energy_true']))

In [144]:
len(MC_flercnn['MCInIcePrimary.energy'][zmask[0]]), len(MC_flercnn['MCInIcePrimary.energy'][emask[0]]), len(MC_flercnn['MCInIcePrimary.energy'][zmask[0] &emask[0]]), len(PD_SANTA['weights'])

(764103, 763942, 763540, 763956)

zen_true,-0.951,0.111,-0.512,...,0.0646,-0.561,-0.601
zen_reco,-0.77,-0.646,0.0287,...,-0.4,-0.289,-0.422
zen_err,-0.18,0.756,-0.541,...,0.465,-0.272,-0.18
energy_true,1.14,3.6,3.01,...,188,84.1,1090
energy_reco,4,7.06,5.47,...,50.8,6.52,70.8
weights,11.3,292,174,...,2.17e+06,183000,6.15e+07
interaction,1,1,1,...,2,2,2
pid,0.445,0.413,0.441,...,0.452,0.698,0.299
event_id,13600,13800,16200,...,129000,130000,13100


In [120]:

same_zenith = pd.DataFrame(MC_flercnn['MCInIcePrimary.dir.coszen'])[0].loc[list(PD_SANTA['zen_true'])]
same_energy = pd.DataFrame(MC_flercnn['MCInIcePrimary.energy']).loc[list(PD_SANTA['energy_true'])]
same_weight = pd.DataFrame(MC_flercnn['I3MCWeightDict.OneWeight']).loc(list(PD_SANTA['weights']))


KeyError: "None of [Index([  -0.9506548438588333,   0.11051429624420825,   -0.5119878311587294,\n       -0.009845490725557515,   -0.8564613398871072,  -0.38795186725475483,\n           0.534265445661094,   -0.3904656587385157,   -0.4523869846558407,\n         -0.7637950646086801,\n       ...\n          -0.632168322697439,   -0.1657415231286018,   -0.9645304827444448,\n          0.4866907196421776,   -0.2923898891034947,   -0.5840337994037508,\n          0.9998026117590023,   0.06457958997092925,   -0.5607492860773015,\n         -0.6014239556968725],\n      dtype='float64', length=2349658)] are in the [index]"

In [117]:
mask = pd.DataFrame(MC_flercnn['MCInIcePrimary.dir.coszen']).isin(list(PD_SANTA['zen_true']))
emask = pd.DataFrame(MC_flercnn['MCInIcePrimary.energy']).isin(list(PD_SANTA['energy_true']))
wmask = pd.DataFrame(MC_flercnn['I3MCWeightDict.OneWeight']).isin(list(PD_SANTA['weights']))
common_event_mask = emask[0] & mask[0] &wmask[0]

In [92]:
PD_flercnn = dm.PointData({'x_true': MC_flercnn['MCInIcePrimary.pos.x'],
                       'x_reco': MC_flercnn['FLERCNN_vertex_x'],
                       'y_true': MC_flercnn['MCInIcePrimary.pos.y'],
                       'y_reco': MC_flercnn['FLERCNN_vertex_y'],
                       'z_true': MC_flercnn['MCInIcePrimary.pos.z'],
                       'z_reco': MC_flercnn['FLERCNN_vertex_z'],
                       'zen_true': MC_flercnn['MCInIcePrimary.dir.coszen'],
                       'zen_reco': MC_flercnn['FLERCNN_coszen'],
                       'energy_true': MC_flercnn['MCInIcePrimary.energy'],
                       'energy_reco': MC_flercnn['FLERCNN_energy'],
                       'weights': MC_flercnn['I3MCWeightDict.OneWeight'],
                       'interaction': MC_flercnn['I3MCWeightDict.InteractionType'],
                       'PID': MC_flercnn['FLERCNN_prob_track'],
                       'muonID': MC_flercnn['FLERCNN_prob_muon_v3'],
                      })[common_event_mask]

In [101]:
cleanup_santa = pd.DataFrame(PD_SANTA['energy_true']).isin(list(PD_flercnn['energy_true']))
#y = MC_flercnn[MC_flercnn['I3EventHeader.event_id'] in PD_SANTA['event_id']]

In [110]:
len(PD_flercnn['weights']), len(PD_SANTA['weights']), len(PD_SANTA['weights'][cleanup_santa[0].values])

(2329918, 2349658, 2329916)

array([ True,  True,  True, ...,  True,  True,  True])

In [89]:
len(MC_flercnn[emask[0] & mask[0] &wmask[0]]['I3EventHeader.event_id'])

2329918

In [ ]:
PUBLIC_PLOTS = True

PERCENTILES = [ 50., 16., 84. ]
PERCENTILE_LINESTYLES = [ "-", "--", "--" ]
PERCENTILE_LABELS = [ "Median", r"$1\sigma$", None ]

TRUE_ENERGY_LABEL = r"E_{\rm{true}}"
RECO_ENERGY_LABEL = r"E_{\rm{reco}}"
ENERGY_UNIT = r"[GeV]"
AZIMUTH_UNIT = r"[rad]"
POSITION_UNIT = r"[m]"
LENGTH_UNIT = r"[m]"
TIME_UNIT = r"[ns]"

In [ ]:
# setup 
fit_dir=None #where is the real data bfp
output_dir='./flercnn_paper_plots/' #where to save plots

# Choose variables of interest
variables = [
    "weights",
    "true_energy",
    "reco_energy",
    "true_coszen",
    "reco_coszen",
#    "reco_z",
    "pid",
]
cnn_variables = [
    "weights",
    "true_energy",
    "reco_energy",
    "true_coszen",
    "reco_coszen",
    "reco_z",

    "reco_rho",

    "l7_muon_classifier_prob_nu",
    "pid",
]

log=False

In [ ]:
veri_physics_params, veri_template_settings, _, veri_template_patches, _ = configure_analysis(
    analysis_name="numu_disappearance", #TODO configurable
    template_settings=VERIFICATION_NEUTRINO_PIPELINE,
)

veri_template_maker, veri_analysis_binning, veri_arrays = get_events(veri_template_settings, param_patches=veri_template_patches, variables=variables)


In [ ]:
#from pisa.core.distribution_maker import DistributionMaker

#template_maker = DistributionMaker([NEUTRINO_PIPELINE])
#veri_template_maker = DistributionMaker([VERIFICATION_NEUTRINO_PIPELINE])

In [ ]:
'''template_maker.run()
veri_template_maker.run()
templates = []
veri_templates = []
for p in template_maker.pipelines :
    #p.data.representation = "events"
    p_data = p.data
    templates.append(p_data)

    
for p in veri_template_maker.pipelines :
    #p.data.representation = "events"
    p_data = p.data
    veri_templates.append(p_data)'''

In [ ]:
'''from pisa.core.container import ContainerSet
out_cs = ContainerSet("template", containers=templates[0])
veri_out_cs = ContainerSet("template", containers=veri_templates[0])'''

In [ ]:
template_patches

In [ ]:
physics_params, template_settings, _, template_patches, _ = configure_analysis(
    analysis_name="numu_disappearance", #TODO configurable
    template_settings=NEUTRINO_PIPELINE,
)

# Get the template (as events)
template_maker, analysis_binning, arrays = get_events(template_settings, param_patches=template_patches, variables=cnn_variables)


In [ ]:
#
# Plot reconstruction performance
# 

# Choose binning
if log :
    energy_bins = np.logspace(np.log10(5.), np.log10(10000.), num=20)
    # energy_bins = np.logspace(np.log10(analysis_binning["pid"].bin_edges.m[0]), np.log10(analysis_binning["pid"].bin_edges.m[-1]), num=21)
    energy_scale = "log"
else :
    energy_bins = get_bins(5., 100., num=24)
    energy_scale = "linear"
    coszen_bins = get_bins(-1., 0., num=24)
    z_bins = get_bins(-1500., -2000., num=50)

In [ ]:
import matplotlib as mpl
TITLE_FONT = 20
def pretty():
    mpl.rcParams['xtick.labelsize'] = TITLE_FONT 
    mpl.rcParams['ytick.labelsize'] = TITLE_FONT 
    mpl.rcParams['axes.labelsize'] = TITLE_FONT 
    mpl.rcParams['axes.titlesize'] = TITLE_FONT 

In [ ]:
def plot_reco_error_vs_energy(
    arrays,
    energy_bins,
    energy_scale,
    error_bins,
    true_key,
    reco_key,
    true_key2,
    reco_key2,
    true_label,
    reco_label,
    true_label2,
    reco_label2,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(7, 5))


        #
        # Get parameters
        #

        # Extract inputs
        true_energy = particle_arrays["true_energy"]
        true_var = particle_arrays[true_key]
        reco_var = particle_arrays[reco_key]
        weights = particle_arrays["weights"]
        true_var2 = particle_arrays[true_key2]
        reco_var2 = particle_arrays[reco_key2]
        
        # Unit conversions
        if rad2deg :
            true_var = np.rad2deg(true_var)
            reco_var = np.rad2deg(reco_var)
            unit = " [deg]"

        # Calc error

        reco_error = reco_var - true_var
        reco_error2 = ( reco_var2 - true_var2 ) / true_var2 # In %
        
        med = np.median(reco_error)
        med2 = np.median(reco_error2)

        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[energy_bins, error_bins], x=true_energy, y=reco_error, weights=weights )
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = energy_bins[0]
        xmax = energy_bins[-1]
        xwidth = ( xmax - xmin ) / 50.
        nx = 25
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_energy,
            y=reco_error,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )
        pc_x2, pc_y2, pc_masks2 = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_energy,
            y=reco_error2,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            y_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=r"$%s$" % (true_label), zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
            ax2.plot(pc_x2, pc_y2[0], color="blue", label="Energy", zorder=5,alpha=0.5)
            ax2.fill_between(pc_x2, pc_y2[1], pc_y2[2], color="blue", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
        patch = mpatches.Patch(color='grey', label=r'1$\sigma$')

        # Fix scale
        ax.set_ylim(-1, 1)
        
        ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        ax2.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=r"True Neutrino Energy %s"%(ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend(handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_error_%s.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [ ]:
def plot_reco_vs_true(
    arrays1,
    arrays2,
    leg1,
    leg2,
    bins,
    scale,
    true_key,
    reco_key,
    true_label,
    reco_label,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays1.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(8, 6))


        #
        # Get parameters
        #

        # Extract inputs
        true_var1 = particle_arrays[true_key]
        reco_var1 = particle_arrays[reco_key]
        weights1 = particle_arrays["weights"]
        
        for particle_key2, particle_arrays2 in arrays2.items() :

            if particle_key2 != "numu_cc" :
                continue  

            # Get parameters
            #

            # Extract inputs
            true_var2 = particle_arrays2[true_key]
            reco_var2 = particle_arrays2[reco_key]
            weights2 = particle_arrays2["weights"]


        
        # Unit conversions
        if rad2deg :
            true_var2 = np.rad2deg(true_var2)
            reco_var2 = np.rad2deg(reco_var2)
            unit = " [deg]"

        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        #ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[bins, bins], x=true_var1, y=reco_var1, weights=weights1)
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = bins[0]
        xmax = bins[-1]
        xwidth = ( xmax - xmin ) / 10.
        nx = 10
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var1,
            y=reco_var1,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )
        print(pc_y)
        pc_x2, pc_y2, pc_masks2 = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var2,
            y=reco_var2,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            plot_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=leg1, zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
            ax.plot(pc_x2, pc_y2[0], color="blue", label=leg2, zorder=5,alpha=0.5)
            ax.fill_between(pc_x2, pc_y2[1], pc_y2[2], color="blue", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
        patch = mpatches.Patch(color='grey', label=r'1$\sigma$')

        # Fix scale
        #ax.set_ylim(-1, 1)
        
        #ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        #hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        #ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        #ax.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(reco_label)
        #r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=true_label, xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend()#handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_vs_true_%s.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [ ]:
plot_reco_vs_true(
    arrays1=arrays,
    arrays2=veri_arrays,
    leg1 = "CNN",
    leg2 = "SANTA",
    bins=energy_bins,
    scale=energy_scale,
    #error_bins=get_bins(-2., 2., num=20),
    true_key='true_energy',
    reco_key="reco_energy",
    true_label="True neutrino energy [GeV]", #r"\cos(\theta_{\rm{zenith}})",
    reco_label="Reco neutrino energy [GeV]", #r"\cos(\theta_{\rm{zenith}})",
    unit=ENERGY_UNIT,
    rad2deg=False,
    output_dir=output_dir,
    include_events_hist=False,
)

In [ ]:
plot_reco_vs_true(
    arrays1=arrays,
    arrays2=veri_arrays,
    leg1 = "CNN",
    leg2 = "SANTA",
    bins=coszen_bins,
    scale=energy_scale,
    #error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_label="True cos(zen)", #r"\cos(\theta_{\rm{zenith}})",
    reco_label="Reco cos(zen)", #r"\cos(\theta_{\rm{zenith}})",
    unit=ENERGY_UNIT,
    rad2deg=False,
    output_dir=output_dir,
    include_events_hist=False,
)

In [ ]:
def plot_reco_vs_true_single(
    arrays1,
    leg1,
    bins,
    scale,
    true_key,
    reco_key,
    true_label,
    reco_label,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays1.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(8, 6))


        #
        # Get parameters
        #

        # Extract inputs
        true_var1 = particle_arrays[true_key]
        reco_var1 = particle_arrays[reco_key]
        weights1 = particle_arrays["weights"]
    
        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        #ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[bins, bins], x=true_var1, y=reco_var1, weights=weights1)
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = bins[0]
        xmax = bins[-1]
        xwidth = ( xmax - xmin ) / 10.
        nx = 10
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var1,
            y=reco_var1,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            plot_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=leg1, zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)

        # Fix scale
        #ax.set_ylim(-1, 1)
        
        #ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        #hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        #ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        #ax.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(reco_label)
        #r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=true_label, xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend()#handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_vs_true_%s_single.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [ ]:
plot_reco_error_vs_energy(
    arrays=arrays,
    energy_bins=energy_bins,
    energy_scale=energy_scale,
    error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_key2="true_energy",
    reco_key2="reco_energy",
    true_label=r"\cos(\theta_{\rm{zenith}})",
    reco_label=r"\cos(\theta_{\rm{zenith}})",
    true_label2=r"True E_{\nu}", #E_{\rm{true}}",
    reco_label2=r"E_{\rm{reco}}",
    unit=ENERGY_UNIT,
    output_dir=output_dir,
    include_events_hist=False,
)

In [ ]:
plot_reco_error_vs_energy(
    arrays=veri_arrays,
    energy_bins=energy_bins,
    energy_scale=energy_scale,
    error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_key2="true_energy",
    reco_key2="reco_energy",
    true_label=r"\cos(\theta_{\rm{zenith}})",
    reco_label=r"\cos(\theta_{\rm{zenith}})",
    true_label2=r"True E_{\nu}", #E_{\rm{true}}",
    reco_label2=r"E_{\rm{reco}}",
    unit=ENERGY_UNIT,
    output_dir=output_dir,
    include_events_hist=False,
)